In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing Libraries

In [0]:
import pandas as pd

Import Dataset

In [3]:

data_temp = pd.read_csv('/content/drive/My Drive/AOBD_code_implementation/Submission_08 5/ID3_data.csv', header=None)
data_temp.head()

,0,1,2,3,4
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes


Give proper name to the columns

In [4]:
data = data_temp.rename(columns={0:'age',1:'income',2:'student',3:'credit_rating',4:'buys_computer'})
data.head()

,age,income,student,credit_rating,buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes


Count number of class of each attribute

In [0]:
class_cnt = []
total_attribute = len(data.columns)
attr_name = data.columns
for i in range(total_attribute):
    class_cnt.insert(i,data[attr_name[i]].unique())
    

In [0]:
suffi_statastic_colname =[]
for i in range(total_attribute-1):
    cnt = len(class_cnt[i])

    for j in range(cnt):
        suffi_statastic_colname.append(attr_name[i]+':'+class_cnt[i][j])


In [7]:
print(suffi_statastic_colname)

['age:youth', 'age:middle_aged', 'age:senior', 'income:high', 'income:medium', 'income:low', 'student:no', 'student:yes', 'credit_rating:fair', 'credit_rating:excellent']


In [8]:
attr_name_class = []

for i in attr_name:
    attr_name_class.append([col for col in suffi_statastic_colname if i in col])

attr_name_class=attr_name_class[:-1]
print(attr_name_class)

attr_name_class = pd.Series(attr_name_class,index=attr_name[:-1])
print(attr_name_class)

[['age:youth', 'age:middle_aged', 'age:senior'], ['income:high', 'income:medium', 'income:low'], ['student:no', 'student:yes'], ['credit_rating:fair', 'credit_rating:excellent']]
age                   [age:youth, age:middle_aged, age:senior]
income                [income:high, income:medium, income:low]
student                              [student:no, student:yes]
credit_rating    [credit_rating:fair, credit_rating:excellent]
dtype: object


In [0]:
import math

def calculate_gain(attr,df_suffi_state,attr_class):
    
    info = 0
    D = sum(df_suffi_state[output_class_name])
    #print('D = ',D)
    for i in range(len(attr_class)):
        Dj = sum(df_suffi_state[attr_class[i]])
        #print('Dj = ',Dj)
        temp_sum=0
        for j in range(len(df_suffi_state)):
            d = df_suffi_state[attr_class[i]][j]
            #print('d = ',d)
            if d==0:
                temp_sum+=0
            else:
                temp_sum += -(d/Dj)*math.log(d/Dj,2)
        
        info += (Dj/D)*temp_sum

    info_D=0
    for j in range(len(df_suffi_state)):
        d_all = df_suffi_state[output_class_name][j]
        #print('d = ',d)
        if d_all==0:
            info_D+=0
        else:
            info_D += -(d_all/D)*math.log(d_all/D,2)

    #print(info)
    return info_D - info


In [10]:
import pandas as pd

suffi_state =[]
output_class = class_cnt[-1]
output_class_name = attr_name[-1]

suffi_state = pd.DataFrame(0,index=output_class, columns=suffi_statastic_colname + [output_class_name])
print(suffi_state)
print(output_class)

     age:youth  age:middle_aged  ...  credit_rating:excellent  buys_computer
no           0                0  ...                        0              0
yes          0                0  ...                        0              0

[2 rows x 11 columns]
['no' 'yes']


In [0]:
def HoeffdingTrees(data, suffi_statastic_colname, output_class_name, n_min, delta, tau):
    # prefix = 'main'
    output = output_class
    
    suffi_state = pd.DataFrame(0,index=output, columns=suffi_statastic_colname + [output_class_name])
    
    # for col in suffi_state.columns:      # suffi_state value zero
    #     suffi_state[col].values[:] = 0

    n = pd.Series([0])
    R = math.log(len(suffi_state), 2)
    for index in range(len(data)):
        # print('suffi_state', suffi_state)
        # print(data.iloc[[i]])
        rec = data.iloc[[index]]
        out_label = rec[output_class_name].values[0]   # no/yes

        suffi_index = list(suffi_state.index.values)
        # if 'main' in prefix:
        for attr in rec:                            # update sufficint statistic
            check = [s for s in suffi_index if attr in s]
            # print('check:',check)
            
            rec_attr_value = rec[attr].values[0]  # youth    

            if len(check)>0:
                out_label = attr+':'+rec_attr_value+':'+out_label
            
            if attr == output_class_name:
                suffi_state[attr][out_label] += 1
                continue
            suffi_state[attr+':'+rec_attr_value][out_label] += 1

            if out_label in suffi_index and len(check)>0:
                n[attr+':'+rec_attr_value] += 1
                index = n[attr+':'+rec_attr_value]
                index_attr = attr+':'+rec_attr_value
        if len(n)==1:
            n[0] += 1
            index = n[0]

        # print(index % n_min, min(suffi_state[attr]))

        if index % n_min == 0 or min(suffi_state[output_class_name]) != 0:
            
            gain_dict = dict()
            for attr in rec:
                if attr == output_class_name:
                    continue
                if len(n) != 1:
                    sub = index_attr    
                    temp_list = [s for s in suffi_index if sub in s]
                    temp_suffi_state = suffi_state.loc[temp_list,:]
                    
                    gain_dict[attr] = calculate_gain(attr, temp_suffi_state, attr_name_class[attr])
                else:
                    gain_dict[attr] = calculate_gain(attr, suffi_state, attr_name_class[attr])
            gain_dict = {k: v for k, v in sorted(gain_dict.items(), key=lambda item: item[1], reverse = True)}  # all attribute gains in reversed order
            gain_attr = list(gain_dict.keys())
            gain_values= list(gain_dict.values())
            # print(gain_dict)
            max_gain_attr = gain_attr[0]
            sec_gain_attr = gain_attr[1]
            gain_diff = gain_values[0] - gain_values[1] # gain difference of top two attribute
            
            epsi = math.sqrt((math.pow(R, 2)*math.log(1/delta))/(2*index))
            # print(index, gain_diff, epsi)
            if gain_diff > epsi or epsi < tau:      # condition of spliting
                print(index, ': split on', max_gain_attr)
                
                suffi_statastic_colname =  list(filter(lambda a: max_gain_attr not in a, suffi_statastic_colname))
                
                attr_list = []
                for attr in attr_name_class[max_gain_attr]:
                    
                    # split_attr = attr.split(":")
                    # prefix = split_attr[1]
                    # print('prefix', prefix)

                    prefix = attr
                    output = ([prefix+':']*2)+output_class
                    temp_suffi_state = pd.DataFrame(0,index=output, columns=suffi_statastic_colname + [output_class_name])
                    # print('temp_suffi_state', temp_suffi_state)
                    frames = [suffi_state, temp_suffi_state]

                    # suffi_state.append(temp_suffi_state)
                    suffi_state = pd.concat(frames)
                    
                    attr_list.append(attr)
                    # print('suffi_state', suffi_state)
                temp_series = pd.Series(0, index=attr_list)
                # print('temp_series', temp_series)
                n=n.add(temp_series, fill_value=0)
    print('n:', n)
    print(suffi_state)
    return suffi_state, n

In [33]:
#              data, n, suffi_statastic_colname, output_class_name, n_min, delta, tau
suffi_state, n = HoeffdingTrees(data, suffi_statastic_colname, output_class_name, 2, 0.2, 0)


3 : split on age
n: 0                  3.0
age:middle_aged    3.0
age:senior         5.0
age:youth          3.0
dtype: float64
                     age:youth  ...  buys_computer
no                         2.0  ...              2
yes                        0.0  ...              1
age:youth:no               NaN  ...              1
age:youth:yes              NaN  ...              2
age:middle_aged:no         NaN  ...              0
age:middle_aged:yes        NaN  ...              3
age:senior:no              NaN  ...              2
age:senior:yes             NaN  ...              3

[8 rows x 11 columns]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [34]:

n

0                  3.0
age:middle_aged    3.0
age:senior         5.0
age:youth          3.0
dtype: float64

In [35]:
suffi_state

,age:youth,age:middle_aged,age:senior,income:high,income:medium,income:low,student:no,student:yes,credit_rating:fair,credit_rating:excellent,buys_computer
no,2.0,0.0,0.0,2,0,0,2,0,1,1,2
yes,0.0,1.0,0.0,1,0,0,1,0,1,0,1
age:youth:no,NaN,NaN,NaN,0,1,0,1,0,1,0,1
age:youth:yes,NaN,NaN,NaN,0,1,1,0,2,1,1,2
age:middle_aged:no,NaN,NaN,NaN,0,0,0,0,0,0,0,0
age:middle_aged:yes,NaN,NaN,NaN,1,1,1,1,2,1,2,3
age:senior:no,NaN,NaN,NaN,0,1,1,1,1,0,2,2
age:senior:yes,NaN,NaN,NaN,0,2,1,1,2,3,0,3


In [15]:
len(data['age'].unique())

3

In [16]:
#len(data[(data['age']=='youth') & (data['buys_computer']=='yes')])
data[(data[data.columns[0]]==data[data.columns[0]].unique()[0]) & (data['buys_computer']=='yes')]

,age,income,student,credit_rating,buys_computer
8,youth,low,yes,fair,yes
10,youth,medium,yes,excellent,yes


In [17]:
str_name = data.columns[0]+'_'+data[data.columns[0]].unique()[0]
print(str_name)
str_split = str_name.split('_')
print(str_split)
data[data[str_split[0]]==str_split[1]]

age_youth
['age', 'youth']


,age,income,student,credit_rating,buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
10,youth,medium,yes,excellent,yes


In [18]:
df['A'][0] += 1

NameError: ignored

In [19]:
df

NameError: ignored

In [0]:
f = data.iloc[[0]]

In [0]:
a = f['age']

In [22]:
print(a.values[0])
'age'+':'+a.values[0]

youth


'age:youth'

In [23]:
rec = data.iloc[[index]]
output_class_name
out_label = rec[output_class_name].values[0]   # no/yes

NameError: ignored

In [24]:
type(output_class)

numpy.ndarray

In [25]:
type((['main_']*2)+output_class)

numpy.ndarray

In [0]:
l=['hello_worlk', 'hello_wlk', 'hello_orlk', 'hello_k', 'arpit', 'tesdt']

In [27]:
'*hello*' in l
sub = 'hello'
print ([s for s in l if sub in s])

['hello_worlk', 'hello_wlk', 'hello_orlk', 'hello_k']


In [28]:
list(filter(lambda a: 'hello' not in a, l))

['arpit', 'tesdt']

In [29]:
df = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
print(df)
df2 = pd.DataFrame([[5], [3]], columns=['A'])
print(df2)
df = df.append(df2, ignore_index=True)
df

   A  B
0  1  2
1  3  4
   A
0  5
1  3


,A,B
0,1,2.0
1,3,4.0
2,5,NaN
3,3,NaN


In [30]:
df['B'][2]+1

nan

In [31]:
sum1

NameError: ignored